In [2]:
from main import*
from functools import partial
import time as tm
m = bi(platform='cpu')

jax.local_device_count 32


# Jax functions

In [3]:
init_key, sample_key = random.split(random.PRNGKey(int(r.randint(0, 10000000))))
init_key = jnp.array(init_key)

@partial(jit, static_argnums=(1, 2,))
def vec_to_mat(arr, N, K):
    return jnp.reshape(arr, (N, K))

@jit
def jax_LinearOperatorDiag(s, cov):    
    def multiply_with_s(a):
        return jnp.multiply(a, s)
    vectorized_multiply = vmap(multiply_with_s)
    return jnp.transpose(vectorized_multiply(cov))
import jax.numpy as jnp

@jit
def diag_pre_multiply(v, m):
    return jnp.matmul(jnp.diag(v), m)

@jit
def random_centered(sigma, cor_mat, offset_mat):
    """Generate the centered matrix of random factors 

    Args:
        sigma (vector): Prior, vector of length N
        cor_mat (2D array): correlation matrix, cholesky_factor_corr of dim N, N
        offset_mat (2D array): matrix of offsets, matrix of dim N*k

    Returns:
        _type_: 2D array
    """
    return jnp.dot(diag_pre_multiply(sigma, cor_mat), offset_mat)



NameError: name 'vmap' is not defined

# Data manipulation

In [ ]:
import numpy as np
def index(df, cols = 'all'):
    index_map = {}
    if cols == 'all':
        colCat = list(df.select_dtypes(['object']).columns)    
        for a in range(len(colCat)):                
            df["index_"+ colCat[a]] =  df.loc[:,colCat[a]].astype("category").cat.codes
            df["index_"+ colCat[a]] = df["index_"+ colCat[a]].astype(np.int64)
            index_map[colCat[a]] = dict(enumerate(df[colCat[a]].astype("category").cat.categories ) )
    else:
        if isinstance(cols, list) == False:
            cols = [cols]
        for a in range(len(cols)):
            df["index_"+ cols[a]] =  df.loc[:,cols[a]].astype("category").cat.codes
            df["index_"+ cols[a]] = df["index_"+ cols[a]].astype(np.int64)
            index_map[cols[a]] = dict(enumerate(df[cols[a]].astype("category").cat.categories ) )
    df.columns = df.columns.str.replace('.', '_')
    df.columns = df.columns.str.replace(' ', '_')
    
    return df

# Rethinking

## 1. Continuous variable: Model (model 4.3)

In [3]:
import time as tm
from main import*
# setup platform------------------------------------------------
m = bi(platform='cpu')

# import data ------------------------------------------------
m.data('/home/sosa/BI/data/Howell1.csv', sep=';') 
m.data = m.data[m.data .age > 18]
m.data.weight = m.data.weight - m.data.weight.mean()
m.data.age = m.data.age - m.data.age.mean()
weight = jnp.array(m.data.weight.values)
height = jnp.array(m.data.height.values)
# TODO: use jax arrays with hugging face package

m.data = dict(height = height, weight = weight)

 # define model ------------------------------------------------
def model(height, weight):
    s = uniform('s', [1], 0, 50)
    a = normal('a', [1], 178, 20)
    b = normal('b', [1], 0, 1)  
    sample("y", Normal(a + b * weight, s), obs=height)

# Run sampler ------------------------------------------------
m.run(model) 
m.sampler.print_summary(0.89)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1000.53it/s, 7 steps of size 7.33e-01. acc. prob=0.92]

BI took: 1.0614 seconds

                mean       std    median      5.5%     94.5%     n_eff     r_hat
      a[0]    154.65      0.28    154.66    154.21    155.07    542.38      1.00
      b[0]      0.90      0.04      0.90      0.84      0.97    532.56      1.00
      s[0]      5.15      0.20      5.14      4.81      5.44    484.57      1.00

Number of divergences: 0


## 2. Categorical variable: Model (model 5.9)

In [ ]:
milk = pd.read_csv("../data/milk.csv", sep=";")
d = milk
d.clade.unique()
d["clade_id"] = d.clade.astype("category").cat.codes
def model(clade_id, K):
    a = numpyro.sample("a", dist.Normal(0, 0.5).expand([len(set(clade_id))]))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = a[clade_id]
    numpyro.sample("height", dist.Normal(mu, sigma), obs=K)
                               


In [138]:
 # setup platform------------------------------------------------
m = bi(platform='cpu')
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/milk.csv', sep=';') 
m.data = index(m.data, cols = "clade")
m.data["K"] = m.data ["kcal_per_g"].pipe(lambda x: (x - x.mean()) / x.std())
index_clade = jnp.array(m.data.index_clade.values, dtype=jnp.int32)
K = jnp.array(m.data.K.values, dtype=jnp.float32)

m.data = dict(K = K, index_clade = index_clade)

 # define model ------------------------------------------------
def model(K, index_clade):
    s = exponential('s', [1], 1)
    a = normal('a', [1], 0, 0.5)
    m = a[index_clade]
    sample("y", Normal(m, s), obs=K)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1076.86it/s, 1023 steps of size 1.09e-03. acc. prob=0.87]

BI took: 1.0159 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.03,0.16,-0.22,0.28
s[0],1.00,0.12,0.84,1.22


## 3. Continuous interactions terms (model 8.3)

In [8]:
 # setup platform------------------------------------------------
m = bi(platform='cpu')
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/tulips.csv', sep=';') 
m.data["blooms_std"] = m.data.blooms /m.data.blooms.max()
m.data["water_cent"] = m.data.water - m.data.water.mean()
m.data["shade_cent"] = m.data.shade - m.data.shade.mean()

water_cent = jnp.array(m.data.water_cent.values)
blooms_std = jnp.array(m.data.blooms_std.values)
shade_cent = jnp.array(m.data.shade_cent.values)


m.data = dict(water_cent = water_cent, blooms_std = blooms_std, shade_cent = shade_cent)

 # define model ------------------------------------------------
def model(blooms_std,water_cent, shade_cent):
    sigma = exponential('sigma', [1], 1)
    bws = normal('bws', [1], 0, 0.25)
    bs = normal('bs', [1], 0, 0.25)
    bw = normal('bw', [1], 0, 0.25)
    a = normal('a', [1], 0.5, 0.25)
    mu = a + bw*water_cent + bs*shade_cent + bws*water_cent*shade_cent
    sample("y", Normal(mu, sigma), obs=blooms_std)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1191.35it/s, 3 steps of size 7.12e-01. acc. prob=0.88]

BI took: 0.9271 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.36,0.03,0.32,0.40
bs[0],-0.11,0.03,-0.16,-0.06
bw[0],0.20,0.04,0.15,0.27
bws[0],-0.14,0.04,-0.20,-0.07
sigma[0],0.14,0.02,0.11,0.17


## 4. Binomial (model 11.1)

In [119]:
import pandas as pd
d = pd.read_csv('/home/sosa/BI/data/chimpanzees.csv', sep = ';')
d["treatment"] = 1 + d.prosoc_left + 2 * d.condition
d["side"] = d.prosoc_left  # right 0, left 1
d["cond"] = d.condition  # no partner 0, partner 1

In [123]:
m = bi()
m.data = dict(pulled_left = jnp.array(d.pulled_left.values, dtype=jnp.int32))
def model(pulled_left):
    a = normal('a', [1], 0, 10)
    sample("y", Binomial(logits=a), obs=pulled_left)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model, init_strategy = numpyro.infer.initialization.init_to_mean()) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1749.34it/s, 1 steps of size 9.55e-01. acc. prob=0.93]

BI took: 0.6800 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.33,0.08,0.19,0.47


##  5. Binomial with indices (model 11.4)

In [29]:
 # setup platform------------------------------------------------
m = bi(platform='cpu')
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/chimpanzees.csv', sep=';') 
m.data["treatment"] = 1 + m.data.prosoc_left + 2 * m.data.condition
treatment = jnp.array(m.data["treatment"], dtype=jnp.int32)
actor = jnp.array(m.data["actor"] )
n_actor = len(jnp.unique(actor))
n_treatment= len(jnp.unique(treatment))
n_actor

jax.local_device_count 32


7

In [33]:
 # setup platform------------------------------------------------
m = bi(platform='cpu')
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/chimpanzees.csv', sep=';') 
m.data["treatment"] = 1 + m.data.prosoc_left + 2 * m.data.condition
treatment = jnp.array(m.data["treatment"], dtype=jnp.int32)
actor = jnp.array(m.data["actor"] )
pulled_left = jnp.array(m.data["pulled_left"] )
n_actor = len(jnp.unique(actor))
n_treatment= len(jnp.unique(treatment))

m.data = dict(
    actor = actor,
    treatment = treatment,
    pulled_left = pulled_left,
    n_actor = n_actor,
    n_treatment = n_treatment
)

def model(n_actor, n_treatment, actor, treatment, pulled_left):
    a = normal('a', [n_actor], 0, 1.5)
    b = normal('b', [n_treatment], 0, 0.5)
    p = a[actor] + b[treatment]
    sample("y", Binomial(1, logits=p), obs=pulled_left)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:03<00:00, 281.56it/s, 1023 steps of size 6.48e-04. acc. prob=0.80]


BI took: 3.6782 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.46,1.30,-1.82,2.12
a[1],0.07,0.26,-0.31,0.52
a[2],3.73,0.45,3.02,4.40
a[3],-0.11,0.20,-0.41,0.22
a[4],-0.01,0.21,-0.32,0.31
a[5],0.10,0.15,-0.15,0.33
a[6],1.69,0.18,1.41,1.99
b[0],0.22,0.46,-0.44,0.97
b[1],-0.62,0.24,-0.97,-0.22
b[2],-0.08,0.24,-0.48,0.23


## 6. Poisson (model 11.10) PB

In [110]:
import tensorflow as tf
import math
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/Kline.csv', sep=';') 
m.data["P"] = m.data.population.apply(math.log).pipe(lambda x: (x - x.mean()) / x.std())
m.data["contact_id"] = (m.data.contact == "high").astype(int)
m.data = dict(total_tools=m.data.total_tools.values, P=m.data.P.values, cid=m.data.contact_id.values)
def model(cid, P, total_tools):
    a = normal('a', [1], 3, 0.5)
    b = normal('b', [1], 0, 0.2)
    l = jnp.exp(a[cid] + b[cid]*P)
    sample("y", Poisson(l), obs=total_tools)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1156.54it/s, 511 steps of size 1.04e-03. acc. prob=0.85]


BI took: 0.9525 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],3.49,0.06,3.40,3.57
b[0],0.34,0.05,0.26,0.41


## 7. Negative binomial (model 11.12) (PB estimation)

In [53]:
import tensorflow_probability.substrates.jax.distributions as tfd
init_key, sample_key = random.split(random.PRNGKey(int(r.randint(0, 10000000))))
init_key = jnp.array(init_key)
num_days = 30
y = tfd.Poisson(rate=1.5).sample(seed = init_key, sample_shape=(num_days,))
num_weeks = 4
y_new = tfd.Poisson(rate=0.5 * 7).sample(seed = init_key, sample_shape=(num_weeks,))
y_all = np.concatenate([y, y_new])
exposure = np.concatenate([np.repeat(1, 30), np.repeat(7, 4)])
monastery = np.concatenate([np.repeat(0, 30), np.repeat(1, 4)])
d = pd.DataFrame.from_dict(dict(y=y_all, days=exposure, monastery=monastery))
d["log_days"] = d.days.pipe(np.log)

/home/sosa/.local/lib/python3.10/site-packages/jax/_src/numpy/array_methods.py:66: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype)
/home/sosa/.local/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/jax/ops.py:339: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return np.array(value, dtype=dtype)
/home/sosa/.local/lib/python3.10/site-packages/tensorflow_probability/python/internal/

In [61]:
d

,y,days,monastery,log_days
0,1.0,1,0,0.00000
1,2.0,1,0,0.00000
2,1.0,1,0,0.00000
3,1.0,1,0,0.00000
4,2.0,1,0,0.00000
5,3.0,1,0,0.00000
6,3.0,1,0,0.00000
7,2.0,1,0,0.00000
8,2.0,1,0,0.00000
9,2.0,1,0,0.00000


In [63]:
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data = dict(
    log_days = jnp.array(d.log_days.values),
    monastery = jnp.array(d.monastery.values),
    output = jnp.array(d.y.values)
)

def model(log_days, monastery, output):
    a = normal('a', [1], 0, 1)
    b = normal('b', [1], 0, 1)
    l = log_days + a +  b * monastery
    sample("y", Poisson(rate = l), obs=output)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


  0%|          | 0/1000 [00:00<?, ?it/s]

sample: 100%|██████████| 1000/1000 [00:00<00:00, 1516.49it/s, 1 steps of size 7.71e-01. acc. prob=0.92]

BI took: 0.7408 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],1.43,0.21,1.07,1.72
b[0],0.28,0.66,-0.80,1.34


## 8. Multinomial (model 11.13)

In [65]:
# simulate career choices among 500 individuals
N = 500  # number of individuals
income = np.array([1, 2, 5])  # expected income of each career
score = 0.5 * income  # scores for each career, based on income

# next line converts scores to probabilities
p = jnp.array(tf.nn.softmax(score))

# now simulate choice
# outcome career holds event type values, not counts
career = tfd.Categorical(probs=p).sample(seed = init_key, sample_shape = N)
result = [income[index] for index in career]
data = {'career': career, 'income': result}
d = pd.DataFrame(data)
career = jnp.array(d.career.values)
career_income = jnp.array(d.income.values)
income = jnp.array(income)
m.data = dict(
    income = income,
    career = career
)

In [68]:
def model(income, career):
    a = normal('a', [2], 0, 1)
    b = halfnormal('b', [1], 0.5)
    s_1 = a[0] + b * income[0]
    s_2 = a[1] + b * income[1]
    s_3 = a[0] + b * income[0]
    p = jax.nn.softmax(jnp.stack([s_1[0], s_2[0], s_3[0]]))
    sample("y", Categorical(probs =  p[career]), obs=career)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

sample: 100%|██████████| 1000/1000 [00:00<00:00, 1015.24it/s, 11 steps of size 1.04e-01. acc. prob=0.93]


BI took: 1.0875 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a[0],0.54,0.70,-0.50,1.61
a[1],-0.56,0.71,-1.65,0.66
b[0],0.34,0.27,0.00,0.71


## 9. Beta binomial (model m12.1)

In [75]:
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/UCBadmit.csv', sep=';') 
m.data["gid"] = (m.data["applicant.gender"] != "male").astype(int)
gid = jnp.array(m.data["gid"].astype('int32').values)
applications = jnp.array(m.data["applications"].astype('float32').values)
admit = jnp.array(m.data["admit"].astype('float32').values)

m.data = dict(
    gid = gid,
    applications = applications,
    admit =  admit
)

def model(gid, applications, admit):
    phi = exponential('phi', [1], 1)
    alpha = normal('alpha', [2], 0., 1.5)
    theta = phi + 2
    pbar = jax.nn.sigmoid(alpha[gid])
    concentration1 = pbar*theta
    concentration0 = (1 - pbar) * theta
    sample("y", BetaBinomial(total_count = applications, concentration1 = concentration1, concentration0 = concentration0), obs=admit)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 804.99it/s, 3 steps of size 6.20e-01. acc. prob=0.86]


BI took: 1.4778 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
alpha[0],-0.40,0.43,-1.09,0.30
alpha[1],-0.34,0.43,-0.96,0.37
phi[0],0.98,0.75,0.00,1.96


## 10. Negative-binomial

In [104]:
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/UCBadmit.csv', sep=';') 
m.data["gid"] = (m.data["applicant.gender"] != "male").astype(int)
gid = jnp.array(m.data["gid"].astype('int32').values)
applications = jnp.array(m.data["applications"].astype('float32').values)
admit = jnp.array(m.data["admit"].astype('float32').values)

m.data = dict(
    gid = gid,
    applications = applications,
    admit =  admit
)

def model(gid, applications, admit):
    phi = exponential('phi', [1], 1)
    alpha = normal('alpha', [2], 0., 1.5)
    theta = phi + 2
    pbar = jax.nn.sigmoid(alpha[gid])
    concentration1 = pbar*theta
    concentration0 = (1 - pbar) * theta
    sample("y", BetaBinomial(total_count = applications, concentration1 = concentration1, concentration0 = concentration0), obs=admit)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 906.62it/s, 3 steps of size 6.20e-01. acc. prob=0.86]


BI took: 1.3320 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
alpha[0],-0.40,0.43,-1.09,0.30
alpha[1],-0.34,0.43,-0.96,0.37
phi[0],0.98,0.75,0.00,1.96


## 11. Zero inflated outcomes

In [139]:
from jax.scipy.special import expit
r.seed(42)
# Define parameters
prob_drink = 0.2  # 20% of days
rate_work = 1     # average 1 manuscript per day

# sample one year of production
N = 365

np.random.seed(365)
drink = np.random.binomial(1, prob_drink, N)
y = (1 - drink) * np.random.poisson(rate_work, N)

# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------

m.data = dict(
    y = jnp.array(y)
)

def model(y):
    al = normal('al', [1], 1, 0.5)
    ap = normal('ap', [1], -1.5, 1)
    p = expit(ap)
    lambda_ = jnp.exp(al)
    sample("y", ZeroInflatedPoisson(p, lambda_), obs=y)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1446.23it/s, 1 steps of size 5.90e-01. acc. prob=0.93]

BI took: 0.7985 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
al[0],0.11,0.08,-0.0,0.25
ap[0],-1.37,0.33,-1.9,-0.85


## 12. OrderedLogistic

In [125]:
import numpyro.distributions as dist
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/Trolley.csv', sep=';') 
d = m.data
# discrete proportion of each response value
pr_k = d.response.value_counts().sort_index().values / d.shape[0]
# cumsum converts to cumulative proportions
cum_pr_k = jnp.cumsum(pr_k, -1)
logit = lambda x: jnp.log(x / (1 - x))  # convenience function
lco = logit(cum_pr_k)

jax.local_device_count 32


In [129]:
import numpyro.distributions as dist
from numpyro.distributions.transforms import OrderedTransform
# setup platform------------------------------------------------
m = bi()
m.data = dict(response = jnp.array(d.response.values - 1))
def model(response):
    cutpoints = numpyro.sample(
        "cutpoints",
        dist.TransformedDistribution(
            dist.Normal(0, 1.5).expand([6]), OrderedTransform()
        ),
    )
    numpyro.sample("R", dist.OrderedLogistic(0, cutpoints), obs=response)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 914.70it/s, 7 steps of size 4.63e-01. acc. prob=0.90] 


BI took: 1.2182 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
cutpoints[0],-1.92,0.03,-1.97,-1.87
cutpoints[1],-1.27,0.02,-1.30,-1.23
cutpoints[2],-0.72,0.02,-0.75,-0.69
cutpoints[3],0.25,0.02,0.22,0.28
cutpoints[4],0.89,0.02,0.86,0.93
cutpoints[5],1.77,0.03,1.73,1.82


## 13. Varying interceps

In [85]:
# setup platform------------------------------------------------
m = bi()
# import data ------------------------------------------------
m.data('/home/sosa/BI/data/reedfrogs.csv', sep=';') 
m.data["tank"] = np.arange(m.data.shape[0])
tank = jnp.array(m.data["tank"].astype('int32').values)
density = jnp.array(m.data["density"].astype('float32').values)
surv = jnp.array(m.data["surv"].astype('int32').values)
m.data = dict(
    tank = tank,
    surv = surv
)

def model(tank, surv):
    sigma = exponential('sigma', [1], 1)
    a_bar = normal('a_bar', [1], 0., 1.5)
    alpha = normal('alpha', [48], a_bar, sigma)
    p = jnp.squeeze(alpha[tank])[0]
    sample("y", Binomial(total_count = density, logits = p), obs=surv)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:00<00:00, 1251.10it/s, 15 steps of size 1.06e-01. acc. prob=0.77]


BI took: 0.9192 seconds


,mean,sd,hdi_5.5%,hdi_94.5%
a_bar[0],0.83,0.90,-0.12,2.42
alpha[0],0.83,0.07,0.72,0.92
alpha[1],0.80,1.55,-1.53,3.38
alpha[2],0.82,1.86,-1.22,3.84
alpha[3],0.83,1.53,-1.36,3.15
alpha[4],0.84,1.78,-2.14,3.17
alpha[5],0.87,1.54,-1.71,3.08
alpha[6],0.77,1.78,-1.66,3.43
alpha[7],0.90,1.52,-1.68,2.92
alpha[8],0.79,1.57,-1.19,3.30


## 13. Varying effects

In [6]:
import numpyro.distributions as dist
a = 3.5  # average morning wait time
b = -1  # average difference afternoon wait time
sigma_a = 1  # std dev in intercepts
sigma_b = 0.5  # std dev in slopes
rho = -0.7  # correlation between intercepts and slopes
Mu = jnp.array([a, b])
cov_ab = sigma_a * sigma_b * rho
Sigma = jnp.array([[sigma_a**2, cov_ab], [cov_ab, sigma_b**2]])
jnp.array([1, 2, 3, 4]).reshape(2, 2).T
sigmas = jnp.array([sigma_a, sigma_b])  # standard deviations
Rho = jnp.array([[1, rho], [rho, 1]])  # correlation matrix

# now matrix multiply to get covariance matrix
Sigma = jnp.diag(sigmas) @ Rho @ jnp.diag(sigmas)

N_cafes = 2000
seed = random.PRNGKey(5)  # used to replicate example
vary_effects = dist.MultivariateNormal(Mu, Sigma).sample(seed, (N_cafes,))
a_cafe = vary_effects[:, 0]
b_cafe = vary_effects[:, 1]

seed = random.PRNGKey(22)
N_visits = 10
afternoon = jnp.tile(jnp.arange(2), N_visits * N_cafes // 2)
cafe_id = jnp.repeat(jnp.arange(N_cafes), N_visits)
mu = a_cafe[cafe_id] + b_cafe[cafe_id] * afternoon
sigma = 0.5  # std dev within cafes
wait = dist.Normal(mu, sigma).sample(seed)
d = pd.DataFrame(dict(cafe=cafe_id, afternoon=afternoon, wait=wait))

In [7]:
# import data ------------------------------------------------
m = bi()
m.data = dict(
    cafe = cafe_id, 
    wait = wait, 
    N_cafes = N_cafes
)
def model(cafe, wait, N_cafes):
    a = numpyro.sample("a", dist.Normal(5, 2))
    b = numpyro.sample("b", dist.Normal(-1, 0.5))
    sigma_cafe = exponential('sigma_cafe', [2], 1)
    sigma = exponential('sigma', [1], 1)
    Rho = numpyro.sample("Rho", dist.LKJ(2, 2))
    cov = jnp.outer(sigma_cafe, sigma_cafe) * Rho
    a_cafe_b_cafe = numpyro.sample(
        "a_cafe,b_cafe", dist.MultivariateNormal(jnp.stack([a, b]), cov).expand([N_cafes])
    )
    a_cafe, b_cafe = a_cafe_b_cafe[:, 0], a_cafe_b_cafe[:, 1]
    mu = a_cafe[cafe] + b_cafe[cafe] * afternoon
    sample("y", Normal(mu, sigma), obs=wait)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 628.21it/s, 15 steps of size 3.00e-01. acc. prob=0.85]


BI took: 4.8678 seconds
BI took: 4.8680 seconds

                         mean       std    median      5.5%     94.5%     n_eff     r_hat
           Rho[0,0]      1.00      0.00      1.00      1.00      1.00       nan       nan
           Rho[0,1]     -0.72      0.15     -0.74     -0.93     -0.50    382.49      1.00
           Rho[1,0]     -0.72      0.15     -0.74     -0.93     -0.50    382.49      1.00
           Rho[1,1]      1.00      0.00      1.00      1.00      1.00     30.82      1.00
                  a      3.11      0.24      3.10      2.75      3.52    521.48      1.00
 a_cafe,b_cafe[0,0]      1.29      0.22      1.30      0.99      1.67    524.79      1.00
 a_cafe,b_cafe[0,1]     -0.07      0.27     -0.08     -0.54      0.30    333.80      1.00
 a_cafe,b_cafe[1,0]      2.61      0.21      2.60      2.28      2.96    462.74      1.00
 a_cafe,b_cafe[1,1]     -0.65      0.23     -0.65     -1.04     -0.32    402.36      1.00
 a_cafe,b_cafe[2,0]      2.57      0.25      2.57  

In [40]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
import numpy as np
#try:
#  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
#except:
#  pass

nest_asyncio.apply()
stan_code = """ 
data{
    int len;
    int N_cafes;
    vector[len] wait;
    array[len] int afternoon;
    array[len] int cafe;
}
parameters{
    vector[N_cafes] b_cafe;
    vector[N_cafes] a_cafe;
    real a;
    real b;
    vector<lower=0>[2] sigma_cafe;
    real<lower=0> sigma;
    corr_matrix[2] Rho;
}
model{
    vector[len] mu;
    Rho ~ lkj_corr( 2 );
    sigma ~ exponential( 1 );
    sigma_cafe ~ exponential( 1 );
    b ~ normal( -1 , 0.5 );    
    a ~ normal( 5 , 2 );
    {
        array[N_cafes] vector[2] YY;
        vector[2] MU;
        MU = [ a , b ]';
        for ( j in 1:N_cafes ) YY[j] = [ a_cafe[j] , b_cafe[j] ]';
        YY ~ multi_normal( MU , quad_form_diag(Rho , sigma_cafe) );
    }
    for ( i in 1:len ) {
        mu[i] = a_cafe[cafe[i]] + b_cafe[cafe[i]] * afternoon[i];        
    }
    
    wait ~ normal( mu , sigma );

}
"""
data = {
    'wait' : d['wait'].values.astype(float),
    'afternoon' : d['afternoon'].values.astype(int),
    'cafe' : d['cafe'].values.astype(int)+1,
    'N_cafes' : N_cafes,
    'len' : len(d['wait'].values)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=1, num_samples=500, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_4314cm3u/model_4xoavcff.stan', line 20, column 4: It
    is suggested to reparameterize your model to replace lkj_corr with
    lkj_corr_cholesky, the Cholesky factor variant. lkj_corr tends to run
    slower, consume more memory, and has higher risk of numerical errors.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/1000)
Sampling:  10% (100/1000)
Sampling:  20% (200/1000)
Sampling:  30% (300/1000)
Sampling:  40% (400/1000)
Sampling:  50% (500/1000)
Sampling:  50% (501/1000)
Sampling:  60% (600/1000)
Sampling:  70% (700/1000)
Sampling:  80% (800/1000)
Sampling:  90% (900/1000)
Sampling: 100% (1000/1000)
Sampling: 100% (1000/1000), done.
Messages received during sampling:
  Gradient evaluation took 0.012288 

Pystan took: 1715.6098 seconds


## 14 Multiple random effects


## 15 Networks

In [4]:
import time as tm
from main import*
# setup platform------------------------------------------------
m = bi(platform='cpu')
kl_dyads  = pd.read_csv('/home/sosa/BI/data/kl_dyads')
d2 = pd.read_csv('/home/sosa/BI/data/kl_households'"", index_col=0)
kl_data = dict(
    N=kl_dyads.shape[0],
    N_households=kl_dyads.hidB.max(),
    did=kl_dyads.did.values - 1,
    hidA=kl_dyads.hidA.values - 1,
    hidB=kl_dyads.hidB.values - 1,
    giftsAB=kl_dyads.giftsAB.values,
    giftsBA=kl_dyads.giftsBA.values,
)
m.data = kl_data

def model(N_households, N, did, hidA, hidB, giftsAB, giftsBA, link=False):
    # gr matrix of varying effects
    Rho_gr = lkj("Rho_gr", [], 2, 4)
    sigma_gr = exponential("sigma_gr", [2], 1)
    cov = jnp.outer(sigma_gr, sigma_gr) * Rho_gr
    gr = multivariatenormal("gr", [N_households], 0, cov)

    # dyad effects
    z = normal("z",[2, N], 0, 1)
    L_Rho_d = lkjcholesky("L_Rho_d",[], 2, 8)
    sigma_d = exponential("sigma_d",[1], 1)
    d = numpyro.deterministic(
        "d", ((jnp.repeat(sigma_d, 2)[..., None] * L_Rho_d) @ z).T
    )

    a = normal('a', [1], 0, 1)
    lambdaAB = jnp.exp(a + gr[hidA, 0] + gr[hidB, 1] + d[did, 0])
    lambdaBA = jnp.exp(a + gr[hidB, 0] + gr[hidA, 1] + d[did, 1])
    sample("giftsAB", Poisson(lambdaAB), obs=giftsAB)
    sample("giftsBA", Poisson(lambdaBA), obs=giftsBA)

    # compute correlation matrix for dyads
    if link:
        numpyro.deterministic("Rho_d", L_Rho_d @ L_Rho_d.T)

# Run sampler ------------------------------------------------

m.run(model) 
m.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:04<00:00, 244.77it/s, 63 steps of size 9.02e-02. acc. prob=0.89]


BI took: 8.1093 seconds

                  mean       std    median      5.5%     94.5%     n_eff     r_hat
L_Rho_d[0,0]      1.00      0.00      1.00      1.00      1.00       nan       nan
L_Rho_d[0,1]      0.00      0.00      0.00      0.00      0.00       nan       nan
L_Rho_d[1,0]      0.88      0.04      0.88      0.82      0.93     53.74      1.02
L_Rho_d[1,1]      0.47      0.07      0.48      0.36      0.57     54.94      1.02
 Rho_gr[0,0]      1.00      0.00      1.00      1.00      1.00       nan       nan
 Rho_gr[0,1]     -0.39      0.20     -0.41     -0.71     -0.08    203.92      1.00
 Rho_gr[1,0]     -0.39      0.20     -0.41     -0.71     -0.08    203.92      1.00
 Rho_gr[1,1]      1.00      0.00      1.00      1.00      1.00    435.00      1.00
        a[0]      0.53      0.17      0.52      0.27      0.79    153.88      1.00
     gr[0,0]     -0.49      0.28     -0.48     -0.91     -0.03    252.20      1.01
     gr[0,1]      0.21      0.22      0.21     -0.15      0.55

In [ ]:
import numpy as onp
import numpyro as numpyro
import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size, print_summary
from numpyro.infer import MCMC, NUTS, Predictive
def model(N_households, N, did, hidA, hidB, giftsAB, giftsBA, link=False):
    # gr matrix of varying effects
    Rho_gr = numpyro.sample("Rho_gr", dist.LKJ(2, 4))
    sigma_gr = numpyro.sample("sigma_gr", dist.Exponential(1).expand([2]))
    cov = jnp.outer(sigma_gr, sigma_gr) * Rho_gr
    gr = numpyro.sample("gr", dist.MultivariateNormal(0, cov).expand([N_households]))

    # dyad effects
    z = numpyro.sample("z", dist.Normal(0, 1).expand([2, N]))
    L_Rho_d = numpyro.sample("L_Rho_d", dist.LKJCholesky(2, 8))
    sigma_d = numpyro.sample("sigma_d", dist.Exponential(1))
    d = numpyro.deterministic(
        "d", ((jnp.repeat(sigma_d, 2)[..., None] * L_Rho_d) @ z).T
    )

    a = numpyro.sample("a", dist.Normal(0, 1))
    lambdaAB = jnp.exp(a + gr[hidA, 0] + gr[hidB, 1] + d[did, 0])
    lambdaBA = jnp.exp(a + gr[hidB, 0] + gr[hidA, 1] + d[did, 1])
    numpyro.sample("giftsAB", dist.Poisson(lambdaAB), obs=giftsAB)
    numpyro.sample("giftsBA", dist.Poisson(lambdaBA), obs=giftsBA)

    # compute correlation matrix for dyads
    if link:
        numpyro.deterministic("Rho_d", L_Rho_d @ L_Rho_d.T)


m14_7 = MCMC(NUTS(model), num_warmup=1000, num_samples=1000, num_chains=4)
m14_7.run(random.PRNGKey(0), **kl_data)

## 16. Gaussian Processes

In [6]:
import time as tm
from main import*
Kline2 = pd.read_csv('/home/sosa/BI/data/Kline2.csv', sep=";")
islandsDistMatrix = pd.read_csv('/home/sosa/BI/data/islandsDistMatrix.csv'"", index_col=0)
d = Kline2
d["society"] = range(1, 11)  # index observations

dat_list = dict(
    T=d.total_tools.values,
    P=d.population.values,
    society=d.society.values - 1,
    Dmat=islandsDistMatrix.values,
)

# setup platform------------------------------------------------
m14_8 = bi(platform='cpu')
m14_8.data = dat_list

@jit
def cov_GPL2(x, sq_eta, sq_rho, sq_sigma):
    N = x.shape[0]
    K = sq_eta * jnp.exp(-sq_rho * jnp.square(x))
    K = K.at[jnp.diag_indices(N)].add(sq_sigma)
    return K


def model(Dmat, P, society, T):
    a = exponential('a', [1], 1)
    b = exponential('b',[1],1)
    g = exponential('g',[1],1)
    etasq = exponential('etasq',[1],2)
    rhosq = exponential('rhosq',[1],0.5)

    # non-centered Gaussian Process prior
    SIGMA = cov_GPL2(Dmat, etasq, rhosq, 0.01)
    L_SIGMA = jnp.linalg.cholesky(SIGMA)
    z = normal('z', [10], 0, 1)
    k = (L_SIGMA @ z[..., None])[..., 0]
    lambda_ = a * P**b / g * jnp.exp(k[society])
    sample("T", Poisson(lambda_), obs=T)

# Run sampler ------------------------------------------------
start = tm.time()    
m14_8.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")
m14_8.sampler.print_summary(0.89)

jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [00:01<00:00, 768.14it/s, 127 steps of size 3.22e-02. acc. prob=0.95]


BI took: 1.9520 seconds

                mean       std    median      5.5%     94.5%     n_eff     r_hat
      a[0]      1.33      0.90      1.11      0.15      2.60    394.53      1.02
      b[0]      0.28      0.09      0.28      0.13      0.41    209.03      1.00
  etasq[0]      0.20      0.20      0.14      0.01      0.42    280.91      1.00
      g[0]      0.59      0.59      0.43      0.02      1.23    302.88      1.02
  rhosq[0]      1.38      1.72      0.75      0.02      3.37    329.98      1.00
      z[0]     -0.49      0.79     -0.46     -1.53      0.84    241.76      1.00
      z[1]      0.37      0.76      0.39     -0.86      1.49    502.49      1.00
      z[2]     -0.26      0.76     -0.21     -1.73      0.68    636.05      1.00
      z[3]      0.93      0.65      0.95     -0.12      1.84    255.45      1.00
      z[4]      0.30      0.61      0.28     -0.52      1.37    330.08      1.00
      z[5]     -1.09      0.72     -1.02     -2.20     -0.05    271.82      1.00
   

In [ ]:

# import data ------------------------------------------------
m.data('/home/sosa/BI/data/Howell1.csv', sep=';') 
m.data = m.data[m.data .age > 18]
m.data.weight = m.data.weight - m.data.weight.mean()
m.data.age = m.data.age - m.data.age.mean()
weight = jnp.array(m.data.weight.values)
height = jnp.array(m.data.height.values)
# TODO: use jax arrays with hugging face package
# TODO: Add scale function
m.data = dict(height = height, weight = weight)

 # define model ------------------------------------------------
def model(height, weight):
    s = uniform('s', [1], 0, 50)
    a = normal('a', [1], 178, 20)
    b = normal('b', [1], 0, 1)  
    sample("y", Normal(a + b * weight, s), obs=height)

# Run sampler ------------------------------------------------
start = tm.time()    
m.run(model) 
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

# Diagnostic ------------------------------------------------
m.summary()



# STRAND

In [66]:
import json
with open('output.json', 'r') as f:
    d = json.load(f)

# Data --------------------------------------------------------------------------------------------
outcomes = jnp.array(d['outcomes']).reshape([50,50])
N_id = outcomes.shape[0]
N_responses = 1
Clique = jnp.array(d['block_set']).reshape([50,2])[:,1:].reshape([1,50])[0]
focal = jnp.array(d['focal_set']).reshape([50,2])[:,1:].reshape([1,50])[0]
target = jnp.array(d['target_set']).reshape([50,2])[:,1:].reshape([1,50])[0]
dyad_set = jnp.array(d['dyad_set']).reshape([50,50])
exposure = jnp.array(d['exposure']).reshape([50,50])
priors = d['priors']
censoring_set = jnp.array(d['censoring_set']).reshape([50,2])[:,1:].reshape([1,50])[0]
detected = jnp.array(d['detected'])
trials = jnp.unique(jnp.array(d['trials']))

# Transformed data --------------------------------------------------------------------------------------------
focal_predictors = 

In [67]:
trials

Array([20], dtype=int32)

In [50]:
d.keys()

dict_keys(['N_networktypes', 'N_id', 'N_responses', 'N_periods', 'N_individual_predictors', 'N_dyadic_predictors', 'outcomes', 'flows', 'individual_predictors', 'dyadic_predictors', 'N_block_predictors', 'N_groups_per_block_type', 'block_predictors', 'outcome_mode', 'exposure', 'N_censoring_predictors', 'detected', 'trials', 'dyad_set', 'focal_set', 'target_set', 'N_params', 'block_set', 'censoring_set', 'N_group_vars', 'N_groups_per_var', 'max_N_groups', 'export_network', 'priors'])

In [1]:
import time as tm
from main import*
# setup platform------------------------------------------------
m = bi(platform='cpu')
def model(N_id, N_responses, N_params, 
          N_group_vars, max_N_groups, N_groups_per_var, block_set,
          focal_set, target_set,dyad_set, 
          outcomes, exposure,
          censoring_set, detected, trials,
          priors, export_network, outcome_mode):
    """_summary_

    Args:
        N_id (_int_): Number of individuals
        N_responses (_int_): Number of outcome networks
        N_params (_1Darray_):  Number of focal, target, and dyadic predictors
        N_group_vars (_int_): Number of block structure variablesNumber of block structure variables
        max_N_groups (_int_): Max number of group labels in any variable
        N_groups_per_var (_1Darray_): Number of group labels, per variable type
        block_set (_1Darray_): Dataframe holding the group ID codes for each person (rows) for each variable type (cols)
        focal_set (_2Darray_): Focal slash decider predictor variables   
        target_set (_2Darray_): Target slash alter predictor variables
        dyad_set (_3Darray_): Dyadic predictor variables
        outcomes (_3Darray_): Outcome network of binary ties
        exposure (_3Darray_): Exposure for each outcome
        censoring_set (_2Darray_):  N_id*N_params censoring
        detected (_1Darray_):
        trials (_1Darray_):
        priors (_2Darray_): Priors in a matrix, see details in the make_priors() function
        export_network (_int_): Controls export of predictions
        outcome_mode (_int_): Are outcomes binomial
    """




jax.local_device_count 32


In [17]:
d.keys()

dict_keys(['N_networktypes', 'N_id', 'N_responses', 'N_periods', 'N_individual_predictors', 'N_dyadic_predictors', 'outcomes', 'flows', 'individual_predictors', 'dyadic_predictors', 'N_block_predictors', 'N_groups_per_block_type', 'block_predictors', 'outcome_mode', 'exposure', 'N_censoring_predictors', 'detected', 'trials', 'dyad_set', 'focal_set', 'target_set', 'N_params', 'block_set', 'censoring_set', 'N_group_vars', 'N_groups_per_var', 'max_N_groups', 'export_network', 'priors'])

In [46]:
outcomes = jnp.array(d['outcomes']).reshape([50,50])
N_id = outcomes.shape[0]
N_responses = 1
Clique = jnp.array(d['block_set']).reshape([50,2])[:,1:].reshape([1,50])[0]
focal = jnp.array(d['focal_set']).reshape([50,2])[:,1:].reshape([1,50])[0]
target = jnp.array(d['target_set']).reshape([50,2])[:,1:].reshape([1,50])[0]
dyad_set = jnp.array(d['dyad_set']).reshape([50,50])


Array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)

# Dirichlet Multinomial with centered random factors

In [6]:
import seaborn as sns
import numpy as np
from jax import random
from jax.nn import softmax
import jax.numpy as jnp
import numpyro as numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

###############################################################################
############ SIMULATING MULTINOMIAL DATA WITH SOFTMAX LINK FUNCTION ###########
def mysoftmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x, axis=0)

K = 3
N = 100
N_obs = 2
sigma_random = 0.6


########################################################
################### Fixed effect Sim ###################
#a = np.random.normal(0, 1, K)
a = np.array([3,1,1]) # Forcing a values


# Factors--------------------------
NY = 4
NV = 8

Y2 = np.full((NV, NY), np.nan) 
means = np.random.normal(0, 1, NY)
offsets = np.random.normal(0, 1, NV)
for i in range(NV):
  for k in range(NY):
    Y2[i,k] = means[k] + offsets[i]

    
b_individual = np.random.normal(0, 1, (N, K))
mu = b_individual + a


# Declare an empty Matrix to fill with data
Y = np.empty((N * N_obs, K))

# Declare an empty vector to fill with IDs
id = []

# Loop over each individual
for i in range(N):
    # Simulate N_obs draws from the multinomial
    Y[i*N_obs:(i+1)*N_obs, :] = np.apply_along_axis(lambda x: np.random.multinomial(100, mysoftmax(x)), 0, mu[i])
    # Assign ID vector
    id += [i] * N_obs


N = N*N_obs
K = K
ni = N
y = jnp.array(Y, dtype=jnp.int32).reshape(N, K)
i_ID = jnp.array(id)

dat = dict(
    K = K,
    ni = ni,
    y = y,
    i_ID = i_ID
)

## Latent variable model

In [3]:
init_key, sample_key = random.split(random.PRNGKey(0))
init_key = jnp.array(init_key)
init_key

Array([4146024105,  967050713], dtype=uint32)

In [ ]:
import pandas as pd 
import math
import os
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import Image, set_matplotlib_formats
from matplotlib.patches import Ellipse, transforms
import jax
import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import expit

import numpy as onp
import numpyro as numpyro

import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size, print_summary
from numpyro.infer import MCMC, NUTS
numpyro.set_platform("cpu")
numpyro.set_host_device_count(30)


# Simulation ---------------
NY = 4
NV = 8

Y2 = np.full((NV, NY), np.nan) 
means = np.random.normal(0, 1, NY)
offsets = np.random.normal(0, 1, NV)
for i in range(NV):
  for k in range(NY):
    Y2[i,k] = means[k] + offsets[i]

b_individual = np.random.normal(0, 1, (N, K))
mu = b_individual + a

Y2 = jnp.array(Y2)



In [14]:

# Building model and sampling it ------------------
def model(NY, NV, Y2):
    means =  numpyro.sample('means', numpyro.distributions.Normal(0,1).expand([NY]))
    offset =  numpyro.sample('offset', numpyro.distributions.Normal(0,1).expand([NV,1]))
    sigma =  numpyro.sample('sigma', numpyro.distributions.Exponential(1).expand([NY])) 
    tmp = jnp.tile(means, (NV, 1)).reshape(NV,NY)  
    mu_l = tmp + offset 
    numpyro.sample('Y2', numpyro.distributions.Normal(mu_l, jnp.tile(sigma, [NV, 1])), obs=Y2)

dat = dict(
    NY = NY,
    NV = NV,
    Y2 = Y2
)
m = MCMC(NUTS(model, init_strategy = numpyro.infer.init_to_median()), num_warmup=1000, num_samples=1000, num_chains=1)
m.run(random.PRNGKey(0), extra_fields=["diverging"], **dat)

sample: 100%|██████████| 2000/2000 [00:01<00:00, 1188.58it/s, 1023 steps of size 1.81e-07. acc. prob=0.74]


In [15]:
import arviz as az
data = az.from_numpyro(m)
az.summary(data)

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
means[0],-1.601,0.0,-1.601,-1.601,0.0,0.0,1000.0,1000.0,NaN
means[1],-0.331,0.0,-0.331,-0.331,0.0,0.0,1000.0,1000.0,NaN
means[2],0.154,0.0,0.154,0.154,0.0,0.0,1000.0,1000.0,NaN
means[3],-0.416,0.0,-0.416,-0.416,0.0,0.0,1000.0,1000.0,NaN
"offset[0, 0]",1.167,0.0,1.167,1.167,0.0,0.0,1000.0,1000.0,NaN
"offset[1, 0]",-0.976,0.0,-0.976,-0.976,0.0,0.0,1000.0,1000.0,NaN
"offset[2, 0]",0.520,0.0,0.520,0.520,0.0,0.0,1000.0,1000.0,NaN
"offset[3, 0]",-0.537,0.0,-0.537,-0.537,0.0,0.0,1000.0,1000.0,NaN
"offset[4, 0]",-0.941,0.0,-0.941,-0.941,0.0,0.0,1000.0,1000.0,NaN
"offset[5, 0]",0.945,0.0,0.945,0.945,0.0,0.0,1000.0,1000.0,NaN


In [16]:
means

array([-1.310413  , -0.0403644 ,  0.44440847, -0.12578016])

## Random centered effects

In [7]:
@jit
def random_centered(sigma, cor_mat, offset_mat):
    """Generate the centered matrix of random factors 

    Args:
        sigma (vector): Prior, vector of length N
        cor_mat (2D array): correlation matrix, cholesky_factor_corr of dim N, N
        offset_mat (2D array): matrix of offsets, matrix of dim N*k

    Returns:
        _type_: 2D array
    """
    return jnp.dot(diag_pre_multiply(sigma, cor_mat), offset_mat)

In [17]:
def model(K, ni, y, i_ID):
    a = normal('a', [K], 0,1)
    Sigma_individual = exponential('Sigma_individual', [ni], 1 )
    L_individual = lkjcholesky('L_individual', [], ni, 1) # Implies a uniform distribution over correlation matrices
    z_individual = normal('z_individual', [ni,K], 0, 1)
    alpha = random_centered(Sigma_individual, L_individual, z_individual)
    lk = jnp.exp(a + alpha[i_ID])
    sample("y", DirichletMultinomial(lk, int(100)), obs=y)

m = bi()
m.data = dat
m.run(model, init_strategy = numpyro.infer.init_to_median(), 
      num_warmup=500, num_samples=500, num_chains=1)


jax.local_device_count 32


sample: 100%|██████████| 1000/1000 [01:47<00:00,  9.27it/s, 35 steps of size 3.91e-07. acc. prob=0.80] 


In [15]:
print('Simulated:')
print(jax.nn.softmax(jnp.array(a))) 
print('Numpypro estimation:')
print(jax.nn.softmax(jnp.mean(jnp.array(m.trace['posterior']['a'][0]), axis = 0)))

Simulated:
[0.786986   0.10650697 0.10650697]
Numpypro estimation:
[0.7328625  0.13674371 0.13039377]


In [ ]:
def model(K, ni, y, i_ID):
    a = normal('a', [K], 0,1)
    Sigma_individual = exponential('Sigma_individual', [ni], 1 )
    L_individual = lkjcholesky('L_individual', [], ni, 1) # Implies a uniform distribution over correlation matrices
    z_individual = normal('z_individual', [ni,K], 0, 1)
    alpha = random_centered2(Sigma_individual, L_individual, z_individual)
    lk = jnp.exp(a + alpha[i_ID])
    sample("y", DirichletMultinomial(lk, int(100)), obs=y)

m = bi()
m.data = dat
m.run(model, init_strategy = numpyro.infer.init_to_median(), 
      num_warmup=500, num_samples=500, num_chains=1)

In [49]:
print('Simulated:')
print(jax.nn.softmax(jnp.array(a))) 
print('Numpypro estimation:')
print(jax.nn.softmax(jnp.mean(jnp.array(m.trace['posterior']['a'][0]), axis = 0)))

Simulated:
[0.786986   0.10650697 0.10650697]
Numpypro estimation:
[0.66280484 0.1774538  0.15974137]


In [ ]:
def model(K, ni, y, i_ID):
    a = normal('a', [K], 0,1)
    Sigma_individual = exponential('Sigma_individual', [ni], 1 )
    L_individual = lkjcholesky('L_individual', [], ni, 1) # Implies a uniform distribution over correlation matrices
    print(L_individual.shape)
    z_individual = normal('z_individual', [ni,K], 0, 1)
    alpha = ((Sigma_individual[..., None] * L_individual) @ z_individual)
    print(alpha.shape)
    lk = jnp.exp(a + alpha[i_ID])
    sample("y", DirichletMultinomial(lk, int(100)), obs=y)

m = bi()
m.data = dat
m.run(model, init_strategy = numpyro.infer.init_to_median(), 
      num_warmup=500, num_samples=500, num_chains=1, chain_method='vectorized')

In [56]:
print('Simulated:')
print(jax.nn.softmax(jnp.array(a))) 
print('Numpypro estimation:')
print(jax.nn.softmax(jnp.mean(jnp.array(m.trace['posterior']['a'][0]), axis = 0)))

Simulated:
[0.786986   0.10650697 0.10650697]
Numpypro estimation:


AttributeError: 'bi' object has no attribute 'trace'

In [56]:
###############################################################################
#################################### Pustan Model  #############################
import time as tm
import stan
import nest_asyncio
import numpy as np
tmp = dat
tmp['y'] = np.array(tmp['y'])
tmp['i_ID'] = np.array(tmp['i_ID']+1)
tmp['ni'] = tmp['ni']
tmp['K'] = tmp['K']
tmp['N'] = int(N)

nest_asyncio.apply()
stan_code = """ 
data {
    int<lower=0>  N;             // number of observations
    int<lower=0>  K;             // number of occupations
    int ni;                     // NUmber of Unique Individauls
    array[N, K] int y;           // array of observed occupation indicators
    array[N]int<lower=0>  i_ID;     // village indicator for each individual
}
parameters {
    vector[K] a;                    // intercept for each occupation
    matrix[ni, K]  z_individual;    // raw random effect for household 
    cholesky_factor_corr[ni] L_individual; // Cholesky factor for 
    vector<lower=0>[ni] Sigma_individual;

}
transformed parameters{
    matrix[ni, K] b_individual;
    b_individual = diag_pre_multiply(Sigma_individual, L_individual) * z_individual;
}
model{
    array[N] vector[K] p;
    matrix[N, K] random_effects;
    to_vector(a) ~ normal(0, 1);
    L_individual ~   lkj_corr_cholesky(2);
    Sigma_individual ~ exponential(1);
    to_vector(z_individual) ~ normal(0, 1);
    // Likelihood for
    for (k in 1:K) {
        for (i in 1:N) {
          random_effects[i, k] = b_individual[i_ID[i], k];
          p[i,k] =  a[k] + random_effects[i, k];
      }
    }
    for (i in 1:(N)) {
        y[i,] ~ dirichlet_multinomial(exp(p[i,]));
    }
}
"""

start = tm.time()
stan_model = stan.build(stan_code, data = tmp)
fit = stan_model.sample(num_chains=1, num_samples=500, num_warmup = 500, init = [{'L_individual': np.zeros((tmp['ni'], tmp['ni']))}])
end = tm.time()    
#df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Sampling:   0%/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

Sampling:   0% (1/1000)
Sampling:  10% (100/1000)
Sampling:  20% (200/1000)
Sampling:  30% (300/1000)
Sampling:  40% (400/1000)
Sampling:  50% (500/1000)
Sampling:  50% (501/1000)
Sampling:  60% (600/1000)
Sampling:  70% (700/1000)
Sampling:  80% (800/1000)
Sampling:  90% (900/1000)
Sampling: 100% (1000/1000)
Sampling: 100% (1000/1000), done.
Messages received during sampling:
  Gradient evaluation took 0.006054 seconds
  1000 transitions using 10 leapfrog steps per transition would take 60.54 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_cholesky_lpdf: Random variable[6] is 0

Pystan took: 432.2593 seconds


In [110]:
print('Simulated:')
print(jax.nn.softmax(jnp.array(np.array([3,1,1])))) 
print('Estimation Multinomial:')
post = m.sampler.get_samples()
print(jax.nn.softmax(jnp.mean(post['a'], axis = 0)))
print('Estimation DirichletMultinomial:')
#post = m2.sampler.get_samples()
#print(jax.nn.softmax(jnp.mean(post['a'], axis = 0)))
df = fit.to_frame()
print('Pytstan estimation')
print(jax.nn.softmax(jnp.array([df['a.1'].mean(),df['a.2'].mean(),df['a.3'].mean()])))

Simulated:
[0.786986   0.10650697 0.10650697]
Estimation Multinomial:
[0.79368174 0.11202765 0.0942907 ]
Estimation DirichletMultinomial:
Pytstan estimation
[0.7914235  0.11247264 0.09610377]


## Gaussian process

In [56]:
# Distance matrix 
L = jnp.arange(1,11)
m = jnp.abs(L[:, None] - L[None, :])

# Kernel
@jit
def sq_exp_gp(m, sq_alpha = .5, sq_rho = .1, delta = 0):
    N = m.shape[0]
    K = jnp.full((N,N),  sq_alpha + delta)
    K = K + sq_alpha * jnp.exp(-sq_rho * m * m )

    mask = jnp.triu(jnp.ones_like(K, dtype=bool))
    cov = jnp.where(mask, K, 0)
    return K, cov

r = sq_exp_gp(m)
Z = Normal(0,1).sample(10)
a = r[1] @ jnp.transpose(r[1]) @ Z

AssertionError: 

In [52]:
L.shape[0]

10

# Model on real data

In [ ]:
def model(K, ni, y, i_ID):
    #individual 
    Sigma_individual = exponential('Sigma_individual', [ni], 1 )
    L_individual = lkjcholesky('L_individual', [], ni, 50)
    z_individual = normal('z_individual', [ni,K], 0, 1)
    alpha_individual = random_centered2(Sigma_individual, L_individual, z_individual)

    #household 
    Sigma_household = exponential('Sigma_household', [ni], 1 )
    L_household = lkjcholesky('L_household', [], ni, 50)
    z_household = normal('z_household', [ni,K], 0, 1)
    alpha_household = random_centered2(Sigma_household, L_household, z_household)

    #village 
    Sigma_village = exponential('Sigma_village', [ni], 1 )
    L_village = lkjcholesky('L_village', [], ni, 50)
    z_village = normal('z_village', [ni,K], 0, 1)
    alpha_village = random_centered2(Sigma_village, L_village, z_village)

    #LK
    random_factors = alpha_individual[i_ID] + alpha_household[i_ID] + alpha_village[i_ID]
    numpyro.sample("y", dist.DirichletMultinomial(a + random_factors[i_ID], int(12083)), obs=y)

# Test parallelized random centered

In [ ]:
init_key, sample_key = random.split(random.PRNGKey(int(r.randint(0, 10000000))))
init_key = jnp.array(init_key)

Sigma_i = Exponential(1).sample(init_key, [ni,])
L_i= LKJCholesky(ni, 1).sample(init_key, [])
z_i= Normal(0, 1).sample([ni,K])
alpha = random_centered(Sigma_i, L_i, z_i)

# Model to latex

In [57]:
import re
from IPython.display import display, Latex
greek_symbols = {
    'alpha': '\\alpha',
    'beta': '\\beta',
    'gamma': '\\gamma',
    'delta': '\\delta',
    'epsilon': '\\epsilon',
    'zeta': '\\zeta',
    'eta': '\\eta',
    'theta': '\\theta',
    'iota': '\\iota',
    'kappa': '\\kappa',
    'lambda': '\\lambda',
    'mu': '\\mu',
    'nu': '\\nu',
    'xi': '\\xi',
    'omicron': 'o',  # No direct LaTeX symbol for omicron, using "o"
    'pi': '\\pi',
    'rho': '\\rho',
    'sigma': '\\sigma',
    'tau': '\\tau',
    'upsilon': '\\upsilon',
    'phi': '\\phi',
    'chi': '\\chi',
    'psi': '\\psi',
    'omega': '\\omega'
}

def convert_to_greek(var_name):
    # Convert variable name to lowercase for case-insensitive matching
    var_name_lower = var_name.lower()
    # Check if the variable name has a corresponding Greek symbol
    if var_name_lower in greek_symbols:
        return greek_symbols[var_name_lower]
    else:
        return var_name

def extract_latex(command):
    # Define a regular expression pattern to match the desired parts of the command
    pattern = r"(\w+)\s*=\s*(\w+)\([^,]+,\s*[^,]+,\s*(.*)\)"
    match = re.match(pattern, command)
    
    if match:
        var_name = match.group(1)
        func_name = match.group(2)
        params = match.group(3)
        # Convert var_name to Greek symbol if applicable
        var_name_latex = convert_to_greek(var_name)
        # Construct the desired LaTeX text
        latex_text = f"{var_name_latex} = {func_name}({params})"
        return latex_text
    else:
        return None
# Example usage
command = "Sigma_i = exponential('Sigma_individual', [ni], 1)"
latex_text = extract_latex(command)
display(Latex(f'''${latex_text}$'''))

<IPython.core.display.Latex object>